# Custom sectionlists in BluePyOpt

- Original idea, usage, expansion and usecase author - Stefano Masoli PhD
- Modification to BluePyOpt code - Werner van Geit (2016)

<div class="alert alert-block alert-info">
This open source software code was developed in part or in whole in the Human Brain Project, funded from the European Union’s Horizon 2020 Framework Programme for Research and Innovation under the Specific Grant Agreement No. 785907 (Human Brain Project SGA2).
</div>

## The code in this usecase is aimed to explain how to expand BluePyOpt to define an Axon Initial Segment and a custom axon.

1. The first few cells provide all the code necessary to create a simple custom axon. *NOTE - This code does not generate an actual output but is for demonstration only.*
1. Following the example, the actual code to simulate a regular granule cell. The multicompartmental model is composed by 4 dendrites, a soma and an axon with an hillock, Axon Initial Segment, ascending axon and two parallel fibers. In total the axon is composed by 311 sections.

# This approach was tested, validated and used in the following papers:
<div class="alert alert-block alert-info">
1) Masoli S., Rizza MF., Sgritta M., Van Geit W., Schürmann F., D’Angelo E.: Single Neuron Optimization as a Basis for Accurate Biophysical Modeling: The Case of Cerebellar Granule Cells. Front. Cell. Neurosci. 2017;11:1–14. <br><br>

Usecase example - __["Optimize a cerebellar granule cell multicompartmental model"](https://collab.humanbrainproject.eu/#/collab/1655/nav/66852)__
<br><br>

2) Masoli S., Tognolina M., Laforenza U., Moccia F., D’Angelo E.: Parameter tuning differentiates granule cell subtypes enriching the repertoire of retransmission properties at the cerebellum input stage. Bioarxiv 2019
<br><br>
This code contained in the second part of this usecase
</div>

# Regular granular cell plot

The model can also be simulated with BlueNaas and is part of Granule cell 2019 live paper: __[HERE](https://blue-naas.humanbrainproject.eu/#/url/cerebellum_circuits/BlueNaaS_models/live_paper_grc/SM_GrC2019_non_adapting_low_ahp.zip)__

it can also be found in the Model Catalog as well as the Knowledgegraph. 

In [ ]:
!pip2 install -q pip wget bluepyopt

In [ ]:
import bluepyopt as bpop
import bluepyopt.ephys as ephys	
import math
import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:
class GrCMorphology2019(ephys.morphologies.NrnFileMorphology):
    def __init__(self, morphology_path, do_replace_axon):

        super(GrCMorphology2019, self).__init__(morphology_path, do_replace_axon)

    @staticmethod
    def replace_axon(sim=None, icell=None):
        """Replace axon"""

        for section in icell.axonal:
            sim.neuron.h.delete_section(sec=section)

        # Create new axon array sections
        total_hill_ais = 2
        total_aa = 18
        total_pf = 291

        #Total_subsections
        total_aa_axon = (total_hill_ais + total_aa)
        total_sections = (total_hill_ais + total_aa + total_pf)
        print 'total_sections',total_sections
        print 'simulation in progress'
        sim.neuron.h.execute('create axon[312]', icell)

        #Hillock
        icell.axon[0].nseg = 1
        icell.axon[0].L = 1
        icell.axon[0].diam = 1.5
        icell.axon[0].Ra = 100
 
        #AIS 
        icell.axon[1].nseg = 1
        icell.axon[1].L = 10
        icell.axon[1].diam = 0.7

        #AA_Axon 
        for i in range(total_hill_ais, (total_aa_axon)):
            icell.axon[i].nseg = 1
            icell.axon[i].L = 7
            icell.axon[i].diam = 0.3

        #PF_axon        
        for x in range(total_aa_axon, (total_sections)):
            icell.axon[x].nseg = 1
            icell.axon[x].L = 7
            icell.axon[x].diam = 0.15

        #axonal section list 
        for z in range(0,(total_sections)):
            icell.axonal.append(sec = icell.axon[i])
            icell.all.append(sec = icell.axon[i])

        #Connections with the soma and the first two sections         
        icell.axon[0].connect(icell.soma[0], 1.0, 0.0)
        icell.axon[1].connect(icell.axon[0], 1.0, 0.0)
        icell.axon[2].connect(icell.axon[1], 1.0, 0.0) 

        #Connections for the ascending axon. Index from 2 to 20
        for m in range(total_hill_ais, total_aa_axon):     
            icell.axon[m+1].connect(icell.axon[m], 1.0, 0.0)
                
        #Connections for the pf_1, Index from 9 to 154  
        for r in range(total_aa_axon, (total_aa_axon + (int(total_pf/2)))):
            icell.axon[r+1].connect(icell.axon[r], 1.0, 0.0)  

        #Connection of the section 155 to the terminal section of the AA.
        icell.axon[(total_aa_axon + (int(total_pf/2)))+1].connect(icell.axon[20], 1.0, 0.0)  

        #Connections for the pf_1, Index from 166 to 311
        for p in range((total_aa_axon + (int(total_pf/2))+1), total_sections):
            icell.axon[p+1].connect(icell.axon[p], 1.0, 0.0)  

        # Section lists for the hilock and axon
        icell.Hilock.append(sec = icell.axon[0])
        icell.AIS.append(sec = icell.axon[1]) 

        # Section list for the aa axon
        for n in range(total_hill_ais, total_aa_axon):
            icell.aa_axon.append(sec = icell.axon[n])

        #Section list for the pd_1 axon
        for g in range(total_aa_axon, (total_aa_axon + (int(total_pf/2)))):
            icell.pf_axon_1.append(sec = icell.axon[g])
  
        #Section list for the pd_2 axon  
        for g2 in range((total_aa_axon + (int(total_pf/2))+1), total_sections):
            icell.pf_axon_2.append(sec = icell.axon[g2])  
       
        sim.neuron.h.celsius = 32

This code is a reduced version of the code above. It containes the critical properties that are required to create a custom axon.
The content can be either included into the main template of a neuron, like in this example, or, to be run on a cluster, saved into a separate file.

```python
#import of BluePyOpt
import bluepyopt as bpop
import bluepyopt.ephys as ephys

#Class that contains the new axon.
class GrCMorphology(ephys.morphologies.NrnFileMorphology):
  def __init__(self, morphology_path, do_replace_axon):

        super(GrCMorphology, self).__init__(morphology_path, do_replace_axon)

  @staticmethod
  def replace_axon(sim=None, icell=None):

        #The number, in square bracket define the number of the sections. 
        #In this case 3 for 3 sections.
        sim.neuron.h.execute('create axon[3]', icell)

        #Hillock
        icell.axon[0].nseg = 1
        icell.axon[0].L = 1.5
        icell.axon[0].diam = 1
        icell.axon[0].Ra = 100
 
        #AIS 
        icell.axon[1].nseg = 1
        icell.axon[1].L = 10
        icell.axon[1].diam = 0.7
        icell.axon[1].Ra = 100
        
        #Axon
        icell.axon[2].nseg = 1
        icell.axon[2].L = 10
        icell.axon[2].diam = 0.7
        icell.axon[2].Ra = 100
        
        icell.Hilock.append(sec = icell.axon[0])
        icell.AIS.append(sec = icell.axon[1]) 
        icell.axon_1.append(sec = icell.axon[2])   
```

In [ ]:
storage = get_bbp_client().document 
storage_path = '58955'
filename = 'GrC_2019_mod_axon.zip'
storage.download_file(os.path.join(storage_path, filename), filename)

In [ ]:
%%capture
!unzip -o GrC_2019_mod_axon.zip

In [ ]:
!rm -rf x86_64
!nrnivmodl >/dev/null 2>/dev/null

### Custom location for the axon

By default a Neuron morphology has the following sectionlists: *somatic, axonal, apical and basal*. 
Let's create an object that points to the somatic sectionlist. This object will be used later to specify where mechanisms have to be added etc.

In [ ]:
#Morphology location
morph = GrCMorphology2019('GrC2018.asc', do_replace_axon = True)

#Locations
dend_loc = ephys.locations.NrnSeclistLocation('basal', seclist_name='basal')
somatic_loc = ephys.locations.NrnSeclistLocation('somatic', seclist_name='somatic')
axon0 = ephys.locations.NrnSeclistLocation('Hilock', seclist_name='Hilock')
axon1 = ephys.locations.NrnSeclistLocation('AIS', seclist_name='AIS')
axon2 = ephys.locations.NrnSeclistLocation('aa_axon', seclist_name='aa_axon')
axon3 = ephys.locations.NrnSeclistLocation('pf_axon_1', seclist_name='pf_axon_1')
axon4 = ephys.locations.NrnSeclistLocation('pf_axon_2', seclist_name='pf_axon_2')

The following code is a reduced version of the one above and it containes the critical parts necessary to define the sectionlist locations. 

```python
#Morphology location and the call to the custom class to replace the axon
morph = GrCMorphology('GrC2017.asc', do_replace_axon = True)

#Locations
#Dendrits sections taken from the morphology
dend_loc = ephys.locations.NrnSeclistLocation('dend', seclist_name='dend')

#Soma section taken from the morphology
somatic_loc = ephys.locations.NrnSeclistLocation('somatic', seclist_name='somatic')

#Axonal custom location. Hilock, AIS and a section for the axon. 
axon0 = ephys.locations.NrnSeclistLocation('Hilock', seclist_name='Hilock')
axon1 = ephys.locations.NrnSeclistLocation('AIS', seclist_name='AIS')
axon2 = ephys.locations.NrnSeclistLocation('axon_1', seclist_name='axon_1')
```

### Ionic channel insertion

In [ ]:
#DENDRITES.

#Passive properties
cm_dend = ephys.parameters.NrnSectionParameter(
        name='cm_dend', 
        param_name='cm',
        value=2.5,
        locations=[dend_loc],
        frozen=True)

cm_soma = ephys.parameters.NrnSectionParameter(
        name='cm_soma', 
        param_name='cm',
        value=2,
        locations=[somatic_loc, axon0],
        frozen=True)

cm_axon_only = ephys.parameters.NrnSectionParameter(
        name='cm_axon', 
        param_name='cm',
        value=1,
        locations=[axon1, axon2,  axon3,  axon4],
        frozen=True)

ena = ephys.parameters.NrnSectionParameter(
        name='ena_all', 
        param_name='ena',
        value=87.39,
        locations=[axon0, axon1, axon2,  axon3,  axon4],
        frozen=True)

ek = ephys.parameters.NrnSectionParameter(
        name='ek_all', 
        param_name='ek',
        value=-88,
        locations=[dend_loc, somatic_loc, axon0, axon1, axon2,  axon3,  axon4],
        frozen=True)

eleak = ephys.parameters.NrnSectionParameter(
        name='eleak_all', 
        param_name='e_Leak',
        value = -60,
        locations=[dend_loc, somatic_loc, axon0, axon1, axon2,  axon3,  axon4],
        frozen=True)

eca = ephys.parameters.NrnSectionParameter(
        name='eca_all', 
        param_name='eca',
        value = 137.5,
        locations=[dend_loc, somatic_loc, axon0, axon1, axon2,  axon3,  axon4],
        frozen=True)

Ra = ephys.parameters.NrnSectionParameter(
        name='ra_all', 
        param_name='Ra',
        value = 100,
        locations=[dend_loc, somatic_loc, axon0, axon1, axon2,  axon3,  axon4],
        frozen=True)

#CHANNELS
#DEND
GrC_Kca1_1_dend = ephys.mechanisms.NrnMODMechanism(
        name='Kca1_1_dend',
        prefix='Kca1_1',
        locations=[dend_loc])

GrC_GRC_CA_dend = ephys.mechanisms.NrnMODMechanism(
        name='GRC_CA_dend',
        prefix='GRC_CA',
        locations=[dend_loc, somatic_loc, axon0, axon1, axon2,  axon3,  axon4])

#SOMA
GrC_Kv4_3_soma = ephys.mechanisms.NrnMODMechanism(
        name='Kv4_3',
        prefix='Kv4_3',
        locations=[somatic_loc])

GrC_Kir2_3_soma = ephys.mechanisms.NrnMODMechanism(
        name='Kir2_3',
        prefix='Kir2_3',
        locations=[somatic_loc])

GrC_Kv1_5_soma = ephys.mechanisms.NrnMODMechanism(
        name='Kv1_5',
        prefix='Kv1_5',
        locations=[somatic_loc])

GrC_Kv2_2_0010_soma = ephys.mechanisms.NrnMODMechanism(
        name='Kv2_2_0010',
        prefix='Kv2_2_0010',
        locations=[somatic_loc])

#Mixed
GrC_Kv1_1 = ephys.mechanisms.NrnMODMechanism(
        name='Kv1_1',
        prefix='Kv1_1',
        locations=[dend_loc, somatic_loc])

GrC_Leak = ephys.mechanisms.NrnMODMechanism(
        name='Leak_all',
        prefix='Leak',
        locations=[dend_loc, somatic_loc, axon0, axon1, axon2,  axon3,  axon4])

GrC_Nav16 = ephys.mechanisms.NrnMODMechanism(
        name='GRC_NA',
        prefix='GRC_NA',
        locations=[axon2,  axon3,  axon4])

GrC_Nav16_FHF = ephys.mechanisms.NrnMODMechanism(
        name='GRC_NA_FHF',
        prefix='GRC_NA_FHF',
        locations=[axon0, axon1])

GrC_Kv3_4 = ephys.mechanisms.NrnMODMechanism(
        name='Kv3_4',
        prefix='Kv3_4',
        locations=[somatic_loc, axon0, axon1, axon2,  axon3,  axon4])

GrC_cdp5_CR = ephys.mechanisms.NrnMODMechanism(
        name='cdp5_CR',
        prefix='cdp5_CR',
        locations=[dend_loc, somatic_loc, axon0, axon1, axon2,  axon3,  axon4])

#AIS
GrC_km_AIS = ephys.mechanisms.NrnMODMechanism(
        name='GRC_KM',
        prefix='GRC_KM',
        locations=[axon1])


### Ionic channels range used in the optimizations

In [ ]:
GrC_Leak_param_dend = ephys.parameters.NrnSectionParameter(                                    
        name='gmax_Leak_dend',
        param_name='gmax_Leak',
        locations=[dend_loc],
        bounds=[0.0002, 0.0004],
        frozen=False)

GrC_Kca1_1_param_dend = ephys.parameters.NrnSectionParameter(
        name='gbar_Kca1_1_dend',
        param_name='gbar_Kca1_1',
        bounds=[0.0010, 0.3],
        locations=[dend_loc],
        frozen=False)

GrC_Cav2_1_param_dend = ephys.parameters.NrnSectionParameter(
        name='gcabar_GRC_CA_dend',
        param_name='gcabar_GRC_CA',
        bounds=[0.002, 0.45],
        locations=[dend_loc],
        frozen=False)

GrC_Kv1_1_param_dend = ephys.parameters.NrnSectionParameter(
        name='gbar_Kv1_1_dend',
        param_name='gbar_Kv1_1',
        bounds=[0.0001, 0.01],
        locations=[dend_loc],
        frozen=False)

#SOMA
GrC_Leak_param_soma = ephys.parameters.NrnSectionParameter(                                    
        name='gmax_Leak_soma',
        param_name='gmax_Leak',
        locations=[somatic_loc],
        bounds=[0.0002, 0.0004],
        frozen=False)  

GrC_Kv3_4_param_soma = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kv3_4_soma',
        param_name='gkbar_Kv3_4',
        locations=[somatic_loc],
        bounds=[0.00005, 0.05], #
        frozen=False) 

GrC_Kv4_3_param_soma = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kv4_3_soma',
        param_name='gkbar_Kv4_3',
        locations=[somatic_loc],
        bounds=[0.002, 0.004],
        frozen=False) 

GrC_Kir2_3_param_soma = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kir2_3_soma',
        param_name='gkbar_Kir2_3',
        locations=[somatic_loc],
        bounds=[0.0005, 0.001],
        frozen=False)

GrC_Cav2_1_param_soma = ephys.parameters.NrnSectionParameter(
        name='gcabar_GRC_CA_soma',
        param_name='gcabar_GRC_CA',
        bounds=[0.0001, 0.007],
        locations=[somatic_loc],
        frozen=False)

GrC_Kv1_1_param_soma = ephys.parameters.NrnSectionParameter(
        name='gbar_Kv1_1_soma',
        param_name='gbar_Kv1_1',
        bounds=[0.003, 0.010],
        locations=[somatic_loc],
        frozen=False)

GrC_Kv1_5_param_soma = ephys.parameters.NrnSectionParameter(
        name='gKur_Kv1_5_soma',
        param_name='gKur_Kv1_5',
        bounds=[0.13195e-4, 0.13195e-2],
        locations=[somatic_loc],
        frozen=False)

GrC_Kv2_2_0010_param_soma = ephys.parameters.NrnSectionParameter(
        name='gKv2_2bar_Kv2_2_0010_soma',
        param_name='gKv2_2bar_Kv2_2_0010',
        bounds=[0.000001, 0.0001],
        locations=[somatic_loc],
        frozen=False)

#AXON
#Hilock
GrC_Leak_param_axon0 = ephys.parameters.NrnSectionParameter(                                    
        name='gmax_Leak_axon0',
        param_name='gmax_Leak',
        locations=[axon0],
        bounds=[0.0002, 0.0004],
        frozen=False) 

GrC_Nav16_param_axon0 = ephys.parameters.NrnSectionParameter(                                    
        name='gnabar_GRC_NA_FHF_axon0',
        param_name='gnabar_GRC_NA_FHF',
        locations=[axon0],
        bounds=[0, 1],
        frozen=False)  

GrC_Kv3_4_param_axon0 = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kv3_4_axon0',
        param_name='gkbar_Kv3_4',
        locations=[axon0],
        bounds=[0.002, 0.8],
        frozen=False) 

GrC_Cav2_1_param_axon0 = ephys.parameters.NrnSectionParameter(
        name='gcabar_GRC_CA_axon0',
        param_name='gcabar_GRC_CA',
        bounds=[0.0001, 0.007],
        locations=[axon0],
        frozen=False)

#AIS
GrC_Leak_param_axon1 = ephys.parameters.NrnSectionParameter(                                    
        name='gmax_Leak_axon1',
        param_name='gmax_Leak',
        locations=[axon1],
        bounds=[0.0002, 0.0004],
        frozen=False) 

GrC_Nav16_param_axon1 = ephys.parameters.NrnSectionParameter(                                    
        name='gnabar_GRC_NA_FHF_axon1',
        param_name='gnabar_GRC_NA_FHF',
        locations=[axon1],
        bounds=[0.1, 3],
        frozen=False)  

GrC_Kv3_4_param_axon1 = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kv3_4_axon1',
        param_name='gkbar_Kv3_4',
        locations=[axon1],
        bounds=[0.0005, 0.4],
        frozen=False) 

GrC_Cav2_1_param_axon1 = ephys.parameters.NrnSectionParameter(
        name='gcabar_GRC_CA_axon1',
        param_name='gcabar_GRC_CA',
        bounds=[0.0001, 0.007],
        locations=[axon1],
        frozen=False)

GrC_km_param_axon1 = ephys.parameters.NrnSectionParameter(
        name='gkbar_GRC_KM_axon1',
        param_name='gkbar_GRC_KM',
        bounds=[0.0001, 0.08],
        locations=[axon1],
        frozen=False)

#aas
GrC_Leak_param_axon2 = ephys.parameters.NrnSectionParameter(                                    
        name='gmax_Leak_axon2',
        param_name='gmax_Leak',
        locations=[axon2],
        bounds=[0.00001, 0.0001],
        frozen=False) 

GrC_Nav16_param_axon2 = ephys.parameters.NrnSectionParameter(                                    
        name='gnabar_GRC_NA_axon2',
        param_name='gnabar_GRC_NA',
        locations=[axon2],
        bounds=[0.005, 1],
        frozen=False)  

GrC_Kv3_4_param_axon2 = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kv3_4_axon2',
        param_name='gkbar_Kv3_4',
        locations=[axon2],
        bounds=[0.0002, 0.05],
        frozen=False) 

GrC_Cav2_1_param_axon2 = ephys.parameters.NrnSectionParameter(
        name='gcabar_GRC_CA_axon2',
        param_name='gcabar_GRC_CA',
        bounds=[0.0001, 0.007],
        locations=[axon2],
        frozen=False)

#Pf1 and Pf2
GrC_Leak_param_axon3 = ephys.parameters.NrnSectionParameter(                                    
        name='gmax_Leak_axon3',
        param_name='gmax_Leak',
        locations=[axon3,axon4],
        bounds=[0.0000001, 0.0000008],
        frozen=False) 

GrC_Nav16_param_axon3 = ephys.parameters.NrnSectionParameter(                                    
        name='gnabar_GRC_NA_axon3',
        param_name='gnabar_GRC_NA',
        locations=[axon3,axon4],
        bounds=[0.005, 1], 
        frozen=False)  

GrC_Kv3_4_param_axon3 = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_Kv3_4_axon3',
        param_name='gkbar_Kv3_4',
        locations=[axon3,axon4],
        bounds=[0.0004, 0.1], 
        frozen=False) 

GrC_Cav2_1_param_axon3 = ephys.parameters.NrnSectionParameter(
        name='gcabar_GRC_CA_axon3',
        param_name='gcabar_GRC_CA',
        bounds=[0.0001, 0.007],
        locations=[axon3,axon4],
        frozen=False)

In [ ]:
GrC2019 = ephys.models.CellModel(
    name='SMGrC2019',
    morph=morph,
    mechs=[
        GrC_Leak,
        GrC_Kca1_1_dend,
        GrC_GRC_CA_dend,
        GrC_Kv1_1,
        GrC_Nav16, 
        GrC_Nav16_FHF, 
        GrC_Kv3_4, 
        GrC_Kv4_3_soma,
        GrC_Kir2_3_soma,
        GrC_Kv1_5_soma,
        GrC_Kv2_2_0010_soma,
        GrC_km_AIS,
        GrC_cdp5_CR
    ],
    params=[
        cm_dend,
        cm_soma,
        cm_axon_only,
        ena,
        ek,
        eleak,
        eca,
        Ra,
        GrC_Leak_param_dend,
        GrC_Kca1_1_param_dend,
        GrC_Cav2_1_param_dend, 
        GrC_Kv1_1_param_dend, 
        GrC_Leak_param_soma, 
        GrC_Kv3_4_param_soma,
        GrC_Kv4_3_param_soma,
        GrC_Kir2_3_param_soma,
        GrC_Cav2_1_param_soma,
        GrC_Kv1_1_param_soma,
        GrC_Kv1_5_param_soma,
        GrC_Kv2_2_0010_param_soma,
        GrC_Leak_param_axon0,
        GrC_Nav16_param_axon0,
        GrC_Kv3_4_param_axon0,
        GrC_Cav2_1_param_axon0,
        GrC_Leak_param_axon1,
        GrC_Nav16_param_axon1,
        GrC_Kv3_4_param_axon1,
        GrC_Cav2_1_param_axon1,
        GrC_km_param_axon1,
        GrC_Leak_param_axon2,
        GrC_Nav16_param_axon2,
        GrC_Kv3_4_param_axon2,
        GrC_Cav2_1_param_axon2,
        GrC_Leak_param_axon3,
        GrC_Nav16_param_axon3,
        GrC_Kv3_4_param_axon3,
        GrC_Cav2_1_param_axon3
    ]
)  

GrC2019.seclist_names.append('Hilock')
GrC2019.seclist_names.append('AIS')
GrC2019.seclist_names.append('aa_axon')
GrC2019.seclist_names.append('pf_axon_1')
GrC2019.seclist_names.append('pf_axon_2')

print GrC2019

### The following code contains the critical parts that are needed to create new sectionslist. When a new sectionlist is added, the name have to be the same as in the morphology class.

```python
GrCmulti = ephys.models.CellModel(
        name='SMGrCmulti',
        morph=morph,
        mechs=[],
	
        params=[])  

#To activate the new sectionlist, present in the custom class, they need to be appended to the rest of the cell.
GrCmulti.seclist_names.append('Hilock')
GrCmulti.seclist_names.append('AIS')
GrCmulti.seclist_names.append('axon_1')
```

Now we add the stimuli, recordings and protocols. For each protocol we add a recording and a stimulus in the soma.

Finally we're at a stage where we can actually run a protocol on the cell. We need to create a Simulator object.

In [ ]:
soma_loc = ephys.locations.NrnSeclistCompLocation(
        name='soma',
        seclist_name='somatic',
        sec_index=0,
        comp_x=0.5)

sweep_protocols = []
for protocol_name, amplitude in [('step1', 0.01), ('step2', 0.016), ('step3', 0.022)]:
    stim = ephys.stimuli.NrnSquarePulse(
                step_amplitude=amplitude,
                step_delay=100,
                step_duration=2000,
                location=soma_loc,
                total_duration=2200)
    rec = ephys.recordings.CompRecording(
            name='%s.soma.v' % protocol_name,
            location=soma_loc,
            variable='v')
    
    protocol = ephys.protocols.SweepProtocol(protocol_name, [stim], [rec])
    sweep_protocols.append(protocol)
threestep_protocol = ephys.protocols.SequenceProtocol('twostep', protocols=sweep_protocols)


#Running a protocol on a cell
nrn = ephys.simulators.NrnSimulator(cvode_active = False)

default_params = {
    'gmax_Leak_dend': 0.00025029700737, 
    'gbar_Kca1_1_dend' : 0.01001807454651,
    'gcabar_GRC_CA_dend': 0.00500128008459, 
    'gbar_Kv1_1_dend' : 0.00381819207934,
    'gmax_Leak_soma' : 0.00029038073716,
    'gkbar_Kv3_4_soma' : 0.00076192450952,  
    'gkbar_Kv4_3_soma' : 0.00281496839061,
    'gkbar_Kir2_3_soma' : 0.00074725514702,
    'gcabar_GRC_CA_soma' : 0.00060938071784,  
    'gbar_Kv1_1_soma' : 0.00569738264555,
    'gKur_Kv1_5_soma' : 0.00083407556714, 
    'gKv2_2bar_Kv2_2_0010_soma' : 1.203410852E-05,
    'gmax_Leak_axon0' : 0.0003695818972,
    'gnabar_GRC_NA_FHF_axon0' : 0.00928805851462,
    'gkbar_Kv3_4_axon0' : 0.02037346310915,
    'gcabar_GRC_CA_axon0' : 0.00057726155447,
    'gmax_Leak_axon1': 0.00029276697557,
    'gnabar_GRC_NA_FHF_axon1': 1.28725006737226,    
    'gkbar_Kv3_4_axon1' : 0.00649595340654,  
    'gcabar_GRC_CA_axon1' : 0.00031198539472,
    'gkbar_GRC_KM_axon1' : 0.00056671971737,
    'gmax_Leak_axon2' : 9.364092125E-05,
    'gnabar_GRC_NA_axon2' : 0.02630163681502, 
    'gkbar_Kv3_4_axon2' : 0.00237386061632,
    'gcabar_GRC_CA_axon2' : 0.00068197420273,
    'gmax_Leak_axon3' : 3.5301616E-07,
    'gnabar_GRC_NA_axon3': 0.01771848449261,
    'gkbar_Kv3_4_axon3': 0.00817568047037,
    'gcabar_GRC_CA_axon3': 0.0002085683353
}

responses = threestep_protocol.run(cell_model=GrC2019, param_values=default_params, sim=nrn)

def plot_responses(responses):
    plt.subplot(3,1,1)
    plt.plot(responses['step1.soma.v']['time'], responses['step1.soma.v']['voltage'], label='step1')
    plt.legend()
    plt.subplot(3,1,2)
    plt.plot(responses['step2.soma.v']['time'], responses['step2.soma.v']['voltage'], label='step2')
    plt.legend()
    plt.subplot(3,1,3)
    plt.plot(responses['step3.soma.v']['time'], responses['step3.soma.v']['voltage'], label='step3')
    plt.legend()
    plt.tight_layout()

plot_responses(responses)
plt.show()